In [2]:
import pandas as pd
import json
import numpy as np
import requests
import time

rec = pd.read_csv('../data/recommendations.csv')


# Column Cleaning Data

In [3]:
# is_recommended only has true
rec = rec.loc[rec['is_recommended']]
# removing unneeded columns
rec = rec[['app_id', 'user_id']]
# Check for Null values in data
print(f"Checking Rec for Nulls ================== \n{rec.isnull().sum()}")

Checking Rec for Nulls ================== 
app_id     0
user_id    0
dtype: int64


# Basic Pandas Overviewing

In [22]:
print(f"Random 5 Values:  ==========\n {rec.sample(5)}")
print(f"Columns: =========== \n {rec.columns}")
print(f"Column Data Types: ============ \n {rec.info()}")
print(f"Shape: ============= \n {rec.shape}")

Random 5 Values:  ==========
            app_id   user_id
17345459   231430   6208732
16719118   629520   1934141
18649600   584400   5557257
19969308   444090   8640524
38121591  1094960  11170906
Columns: =========== 
 Index(['app_id', 'user_id'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 35304398 entries, 0 to 41154793
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   app_id   int64
 1   user_id  int64
dtypes: int64(2)
memory usage: 808.1 MB
Column Data Types: ============ 
 None
Shape: ============= 
 (35304398, 2)


# Data Overview

In [23]:
unique_users = rec["user_id"].nunique()
unique_games = rec["app_id"].nunique()
total_len = len(rec)
print(f"Unique Users: =============== \n {unique_users} of {total_len} | {unique_users / total_len * 100}%")
print(f"Unique Games: =============== \n {unique_games} of{total_len} | {unique_games / total_len * 100}%")

# Value Counts:
user_activity = rec['user_id'].value_counts()
print(f"User Value Count: ====================== \n {user_activity}")
game_popularity = rec['app_id'].value_counts()
print(f"Game Value Count: ====================== \n {game_popularity}")

# Users: How many games per user?
print("Games per user - statistics: ====================== \n ")
print(user_activity.describe())


# Games: How many users per game?
print("\nUsers per game - statistics: ====================== \n ")  
print(game_popularity.describe())

# Find most active users and popular games
print(f"\nMost active user has {user_activity.max()} interactions")
print(f"Most popular game has {game_popularity.max()} users")

# User Distribution
print("User activity distribution: ====================== \n ")
print(f"Users with 1 game: ====================== \n {(user_activity == 1).sum():,}")
print(f"Users with 2-4 games: ====================== \n {((user_activity >= 2) & (user_activity <= 4)).sum():,}")
print(f"Users with 5+ games: ====================== \n {(user_activity >= 5).sum():,}")
print(f"Users with 10+ games: ====================== \n {(user_activity >= 10).sum():,}")

# Game Distribution
print("\nGame popularity distribution: ====================== \n ")
print(f"Games with 1 user: ====================== \n {(game_popularity == 1).sum():,}")
print(f"Games with 2 users: ====================== \n {((game_popularity == 2)).sum():,}")
print(f"Games with 3 users: ====================== \n {(game_popularity == 3).sum():,}")
print(f"Games with 4 users: ====================== \n {(game_popularity == 4).sum():,}")
print(f"Games with 50+ users: ====================== \n {(game_popularity >= 50).sum():,}")

Unique Users: =============== 
 12663134 of 35304398 | 35.8684320293466%
Unique Games: =============== 
 37419 of35304398 | 0.10598962769454388%
User Value Count: ====================== 
 user_id
5112758     3920
5669734     3479
11553593    3392
5390510     3023
1353327     2262
            ... 
10056071       1
10310667       1
7057335        1
13132997       1
2306695        1
Name: count, Length: 12663134, dtype: int64
Game Value Count: ====================== 
 app_id
440        294879
252490     226293
292030     193139
431960     186792
730        186306
            ...  
1554670         1
1098790         1
886670          1
1840510         1
1687020         1
Name: count, Length: 37419, dtype: int64
Games per user - statistics: ====================== 
 
count    1.266313e+07
mean     2.787967e+00
std      6.757650e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      3.920000e+03
Name: count, dtype: float64

Users per game - statist

# Data Cleaning

In [ ]:
# Filter 1: Remove users with too few games
min_user_games = 5  # Users must have recommended at least 5 games
filtered_active_users = user_activity[user_activity >= min_user_games].index
filtered_rec = rec[rec['user_id'].isin(filtered_active_users)]

# Filter 2: Remove unpopular games  
min_game_users = 15  # Games must have at least 15 users
filtered_game_popularity = filtered_rec['app_id'].value_counts()
popular_games = filtered_game_popularity[filtered_game_popularity >= min_game_users].index
filtered_rec = filtered_rec[filtered_rec['app_id'].isin(popular_games)]

# Value Counts:
filtered_user_activity = filtered_rec['user_id'].value_counts()
filtered_game_popularity = filtered_rec['app_id'].value_counts()
print(f"User Distribution: ====================== \n {filtered_user_activity.describe()}")
print(f"Game Distribution: ====================== \n {filtered_game_popularity.describe()}")

# User Distribution
print("User activity distribution: ====================== \n ")
print(f"Users with 1-3 games: ====================== (Values Are From Game Filter Downgrade)\n {((filtered_user_activity >= 1) & (filtered_user_activity <= 3)).sum():,}")
print(f"Users with 4+ games: ====================== \n {(filtered_user_activity >= 4).sum():,}")

# Game Distribution
print("\nGame popularity distribution: ====================== \n ")
print(f"Games with 1-9 users: ====================== \n {((filtered_game_popularity >= 1) & (filtered_game_popularity < 10)).sum():,}")
print(f"Games with 10+ users: ====================== \n {(filtered_game_popularity >= 10).sum():,}")

rec = filtered_rec.copy()


User Distribution: ====================== 
 count    1.624058e+06
mean     1.105287e+01
std      1.554495e+01
min      1.000000e+00
25%      6.000000e+00
50%      7.000000e+00
75%      1.100000e+01
max      3.073000e+03
Name: count, dtype: float64
Game Distribution: ====================== 
 count     22214.000000
mean        808.071441
std        3707.323929
min          15.000000
25%          29.000000
50%          71.000000
75%         264.000000
max      115244.000000
Name: count, dtype: float64
User activity distribution: ====================== 
 
Users with 1-3 games: ====================== (Values Are From Game Filter Downgrade)
 374
Users with 4+ games: ====================== 
 1,623,684

Game popularity distribution: ====================== 
 
Games with 1-9 users: ====================== 
 0
Games with 10+ users: ====================== 
 22,214


# Query Around

In [25]:
print(f"Finding Which User Recommended Team Fortress 2 ============================\n{rec.query('app_id == 440')}")
print(f"Shape of it ============================\n{rec.query('app_id == 440').shape}")

Finding Which User Recommended Team Fortress 2 ============================
          app_id   user_id
7102         440   4327329
12403        440  11692274
13725        440  11216854
14766        440  12522295
18053        440  12052932
...          ...       ...
33843171     440   3633082
33854538     440  11431931
33866199     440   7844778
33866861     440   5643333
33912772     440     27749

[115244 rows x 2 columns]
Shape of it ============================
(115244, 2)


# Getting SteamSpy Game Data

In [26]:
def collect_steamspy_data_paginated(target_game_ids, max_pages=None, delay=1.0, start_page=0):
    """
    Collect SteamSpy data using paginated API calls
    
    Args:
        target_game_ids: set of app_ids we need metadata for
        max_pages: maximum pages to fetch (None for all)
        delay: seconds between requests
        start_page: starting page number (default 0)
    """
    collected_games = {}
    found_target_games = set()
    page = start_page - 1  # Will be incremented to start_page in first iteration
    
    print(f"Looking for {len(target_game_ids):,} target games...")
    print(f"Starting paginated collection from page {start_page}...")
    
    while True:
        page += 1
        print(f"\nProcessing page {page}...")
        
        try:
            # Make API request
            url = f"https://steamspy.com/api.php?request=all&page={page}"
            response = requests.get(url, timeout=15)
            
            if response.status_code != 200:
                print(f"HTTP {response.status_code} on page {page}")
                if response.status_code == 429:  # Rate limited
                    print("Rate limited - waiting 10 seconds...")
                    time.sleep(10)
                    continue
                else:
                    break
                
            # Parse JSON response
            page_data = response.json()
            
            # Check if page is empty (end of data)
            if not page_data or len(page_data) == 0:
                print("Empty page - reached end of data")
                break
                
            print(f"Got {len(page_data)} games on page {page}")
            
            # Filter for games
            page_matches = 0
            for app_id_str, game_data in page_data.items():
                try:
                    app_id_int = int(app_id_str)
                    if app_id_int in target_game_ids:
                        # Only keep games with essential data
                        if game_data.get('name') and game_data.get('name').strip():
                            collected_games[app_id_str] = game_data
                            found_target_games.add(app_id_int)
                            page_matches += 1
                except (ValueError, TypeError):
                    continue  # Skip invalid app_ids
            
            print(f"Found {page_matches} target games with valid data on this page")
            print(f"Total collected: {len(collected_games):,}")
            print(f"Progress: {len(found_target_games)}/{len(target_game_ids)} ({len(found_target_games)/len(target_game_ids)*100:.1f}%)")
            
            # Save progress periodically
            if page % 20 == 0:
                with open(f'../data/steamspy_progress_page_{page}.json', 'w') as f:
                    json.dump(collected_games, f)
                print(f"Saved progress at page {page}")
                
            # Respect max_pages limit
            if max_pages and page >= max_pages:
                print(f"Reached max_pages limit ({max_pages})")
                break
                
        except Exception as e:
            print(f"Error on page {page}: {e}")
            print("Waiting 5 seconds before retry...")
            time.sleep(5)
            continue
            
        # Be respectful to the API
        time.sleep(delay)
    
    print(f"\n=== COLLECTION COMPLETE ===")
    print(f"Pages processed: {page}")
    print(f"Games collected: {len(collected_games):,}")
    print(f"Target games found: {len(found_target_games):,}/{len(target_game_ids):,}")
    print(f"Final coverage: {len(found_target_games)/len(target_game_ids)*100:.1f}%")
    
    return collected_games, found_target_games

In [27]:
target_game_ids = set(rec['app_id'].unique())
print(f"Target games needed: {len(target_game_ids):,}")

# Full collection
full_data, found_games = collect_steamspy_data_paginated(
    target_game_ids=target_game_ids,
    max_pages=100,
    start_page=0
)

Target games needed: 22,214
Looking for 22,214 target games...
Starting paginated collection from page 0...

Processing page 0...
Got 1000 games on page 0
Found 851 target games with valid data on this page
Total collected: 851
Progress: 851/22214 (3.8%)
Saved progress at page 0

Processing page 1...
Got 1000 games on page 1
Found 817 target games with valid data on this page
Total collected: 1,668
Progress: 1668/22214 (7.5%)

Processing page 2...
Got 1000 games on page 2
Found 809 target games with valid data on this page
Total collected: 2,477
Progress: 2477/22214 (11.2%)

Processing page 3...
Got 1000 games on page 3
Found 783 target games with valid data on this page
Total collected: 3,259
Progress: 3259/22214 (14.7%)

Processing page 4...
Got 1000 games on page 4
Found 749 target games with valid data on this page
Total collected: 4,007
Progress: 4007/22214 (18.0%)

Processing page 5...
Got 1000 games on page 5
Found 756 target games with valid data on this page
Total collected: 4

# Further Data Cleaning for Item Sparse Matrix Later

In [28]:
# Further Data Cleaning for Item-Based Sparse Matrix
if full_data:
    print(f"\nCollection successful: {len(full_data):,} games with metadata")
    
    # Clean dataset for item-based recommendations
    print("\nCreating clean dataset for item-based collaborative filtering")
    
    # Convert SteamSpy data to DataFrame
    steamspy_df = pd.DataFrame.from_dict(full_data, orient='index')
    steamspy_df = steamspy_df.reset_index()
    steamspy_df = steamspy_df.rename(columns={'index': 'app_id'})
    steamspy_df['app_id'] = steamspy_df['app_id'].astype(int)
    
    print(f"SteamSpy DataFrame shape: {steamspy_df.shape}")
    
    # Define essential columns for item-based recommendations
    essential_columns = ['app_id', 'name', 'developer', 'publisher']
    
    # Keep all available columns that might be useful
    available_cols = [col for col in essential_columns if col in steamspy_df.columns]
    steamspy_clean = steamspy_df[available_cols].copy()
    
    print(f"Available metadata columns: {available_cols}")
    
    # Filter interactions and merge with metadata
    print(f"\nFiltering and merging data for item-based system...")
    
    # Only keep games that have complete metadata
    games_with_metadata = set(steamspy_clean['app_id'].unique())
    print(f"Games with SteamSpy metadata: {len(games_with_metadata):,}")
    
    # Filter recommendations to only include games with metadata
    rec_filtered = rec[rec['app_id'].isin(games_with_metadata)].copy()
    print(f"Interactions with metadata: {len(rec_filtered):,} (was {len(rec):,})")
    print(f"Reduction: {len(rec) - len(rec_filtered):,} interactions removed")
    
    # Re-apply filtering after metadata merge (some games might have been lost)
    print(f"\nRe-applying user/game thresholds after metadata filtering...")
    
    # Filter 1: Remove users with too few games (after metadata filtering)
    min_user_games = 5
    post_metadata_user_activity = rec_filtered['user_id'].value_counts()
    active_users = post_metadata_user_activity[post_metadata_user_activity >= min_user_games].index
    rec_filtered = rec_filtered[rec_filtered['user_id'].isin(active_users)]
    print(f"Users after {min_user_games}+ games filter: {rec_filtered['user_id'].nunique():,}")
    
    # Filter 2: Remove unpopular games (after user filtering)
    min_game_users = 15
    post_user_filter_game_popularity = rec_filtered['app_id'].value_counts()
    popular_games = post_user_filter_game_popularity[post_user_filter_game_popularity >= min_game_users].index
    rec_filtered = rec_filtered[rec_filtered['app_id'].isin(popular_games)]
    print(f"Games after {min_game_users}+ users filter: {rec_filtered['app_id'].nunique():,}")
    
    # Merge with metadata (inner join for complete data only)
    rec_enhanced = rec_filtered.merge(
        steamspy_clean,
        on='app_id',
        how='inner'  # Only keep games with complete metadata
    )
    
    # Final data quality cleaning
    print(f"\nFinal data quality cleaning...")
    before_cleaning = len(rec_enhanced)
    
    # Remove rows where essential data is missing or empty
    rec_enhanced = rec_enhanced[rec_enhanced['name'].notna()]
    rec_enhanced = rec_enhanced[rec_enhanced['name'].str.strip() != '']
    
    if 'developer' in rec_enhanced.columns:
        rec_enhanced = rec_enhanced[rec_enhanced['developer'].notna()]
        rec_enhanced = rec_enhanced[rec_enhanced['developer'].str.strip() != '']
    
    after_cleaning = len(rec_enhanced)
    print(f"Removed {before_cleaning - after_cleaning:,} rows with missing essential data")
    
    # Final statistics optimized for item-based filtering
    print(f"\nFINAL CLEAN DATASET STATISTICS (Item-Based Ready):")
    print(f"Total interactions: {len(rec_enhanced):,}")
    print(f"Unique users: {rec_enhanced['user_id'].nunique():,}")
    print(f"Unique games: {rec_enhanced['app_id'].nunique():,}")
    print(f"Average games per user: {len(rec_enhanced) / rec_enhanced['user_id'].nunique():.1f}")
    print(f"Average users per game: {len(rec_enhanced) / rec_enhanced['app_id'].nunique():.1f}")
    print(f"Data completeness: 100% (all rows have metadata)")
    
    # Calculate sparsity for item-based matrix (games × users)
    n_games = rec_enhanced['app_id'].nunique()
    n_users = rec_enhanced['user_id'].nunique()
    sparsity = (1 - len(rec_enhanced) / (n_games * n_users)) * 100
    print(f"Matrix sparsity: {sparsity:.2f}% (Item-User matrix: {n_games:,} × {n_users:,})")
    
    # Memory estimation for item-user sparse matrix
    estimated_sparse_mb = (len(rec_enhanced) * 12) / (1024**2)  # 12 bytes per nnz element in CSR
    print(f"Estimated sparse matrix memory: ~{estimated_sparse_mb:.1f} MB")
    
    # Show sample of clean data
    print(f"\nSample clean data for item-based recommendations:")
    display_cols = ['user_id', 'app_id', 'name', 'developer']
    available_display_cols = [col for col in display_cols if col in rec_enhanced.columns]
    print(rec_enhanced[available_display_cols].head())
    
    # Save datasets optimized for item-based collaborative filtering
    print(f"\nSaving clean datasets for item-based system...")
    
    # 1. Core interaction data for sparse matrix creation (MOST IMPORTANT)
    interaction_data = rec_enhanced[['user_id', 'app_id']].copy()
    # Add implicit rating (1 = user played this game)
    interaction_data['rating'] = 1
    interaction_data.to_csv('../data/clean_interactions.csv', index=False)
    print("Saved: clean_interactions.csv (Ready for sparse matrix creation)")
    
    # 2. Game metadata for recommendation display
    game_metadata = rec_enhanced[available_cols].drop_duplicates(subset=['app_id']).copy()
    game_metadata = game_metadata.sort_values('app_id').reset_index(drop=True)
    game_metadata.to_csv('../data/game_metadata.csv', index=False)
    print("Saved: game_metadata.csv")
    
    # 3. Full enhanced dataset (backup)
    rec_enhanced.to_csv('../data/recommendations_enhanced.csv', index=False)
    print("Saved: recommendations_enhanced.csv (Complete dataset backup)")
    
    # 4. Save the raw SteamSpy data for future use
    with open('../data/steamspy_collected_data.json', 'w') as f:
        json.dump(full_data, f, indent=2)
    print("Saved: steamspy_collected_data.json (Raw API data)")
    
    # Summary for next steps
    print(f"clean_interactions.csv contains {len(interaction_data):,} user-game-rating tuples")
    print(f"Matrix dimensions will be: {n_games:,} games × {n_users:,} users")
    print(f"Estimated memory usage: ~{estimated_sparse_mb:.1f} MB (sparse)")
    print(f"All metadata is complete for human-readable recommendations")
    
    # Update the main rec DataFrame
    rec = rec_enhanced.copy()
else:
    print("Collection failed - cannot proceed with matrix creation")


Collection successful: 20,305 games with metadata

Creating clean dataset for item-based collaborative filtering
SteamSpy DataFrame shape: (20305, 18)
Available metadata columns: ['app_id', 'name', 'developer', 'publisher']

Filtering and merging data for item-based system...
Games with SteamSpy metadata: 20,305
Interactions with metadata: 17,582,388 (was 17,950,499)
Reduction: 368,111 interactions removed

Re-applying user/game thresholds after metadata filtering...
Users after 5+ games filter: 1,579,456
Games after 15+ users filter: 20,209

Final data quality cleaning...
Removed 9,888 rows with missing essential data

FINAL CLEAN DATASET STATISTICS (Item-Based Ready):
Total interactions: 17,398,025
Unique users: 1,579,456
Unique games: 20,156
Average games per user: 11.0
Average users per game: 863.2
Data completeness: 100% (all rows have metadata)
Matrix sparsity: 99.95% (Item-User matrix: 20,156 × 1,579,456)
Estimated sparse matrix memory: ~199.1 MB

Sample clean data for item-bas